In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os
import shutil
import warnings

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
eng = nivapy.da.connect_postgis()

Connection successful.


In [4]:
year = 2022
sector = "large wastewater"

# Vassdragsområder draining to Trondheimsfjorden
vassoms = range(120, 133)

cap_thresh = 10000

In [5]:
# Get data from database
df = teo.io.get_raw_annual_point_data(
    eng,
    year,
    sector,
    par_list=[
        "totn_kg",
        "din_kg",
        "ton_kg",
        "totp_kg",
        "tdp_kg",
        "tpp_kg",
        "toc_kg",
        "ss_kg",
    ],
)
df["vassom"] = df["regine"].str.split(".", n=1, expand=True)[0]
vassom_list = [f"{vassom:03d}" for vassom in vassoms]
df = df.query("vassom in @vassom_list")
cols = ["vassom", "site_id", "site_name", "type", "TOTP_kg", "TOTN_kg"]
df = df[cols]

# Get capacity info
cap_xlsx = f"/home/jovyan/shared/common/teotil3/point_data/{year}/avlop_stor_anlegg_{year}_treatment_types_bof_kof.xlsx"
cap_df = pd.read_excel(cap_xlsx, sheet_name="data")[["ANLEGGSNR", "KAPASITET1"]]
cap_df.columns = ["site_id", "capacity"]

# Get efficiencies
eff_xlsx = f"/home/jovyan/shared/common/teotil3/point_data/{year}/avlop_stor_anlegg_{year}_raw.xlsx"
eff_df = pd.read_excel(eff_xlsx, sheet_name=f"store_anlegg_{year}")[
    ["ANLEGGSNR", "RENSEEFFEKT_P", "RENSEEFFEKT_N"]
]
eff_df.columns = ["site_id", "renseeffekt_p", "renseeffekt_n"]

# Join
df = pd.merge(df, cap_df, how="left", on="site_id")
df = pd.merge(df, eff_df, how="left", on="site_id")

# Just sites above capacity threshold
df = (
    df.query("capacity > @cap_thresh")
    .sort_values("capacity", ascending=False)
    .reset_index(drop=True)
)

df

,vassom,site_id,site_name,type,TOTP_kg,TOTN_kg,capacity,renseeffekt_p,renseeffekt_n
0,123,1601AL36,Høvringen renseanlegg,Annen rensing,60300.6,422313.6,170000.0,0.23,0.20
1,123,1601AL00,Ladehammeren renseanlegg,Kjemisk,27534.0,196717.4,122000.0,0.30,0.20
2,124,1714AL31,Stjørdal ARA renseanlegg,Kjemisk,360.6,79475.3,35000.0,0.96,0.15
3,128,1702AL38,Løsberga renseanlegg,Kjemisk,1334.3,61831.6,32000.0,0.87,0.20
4,126,1721AL01,Ørin renseanlegg,Kjemisk,2892.1,36864.6,28000.0,0.86,0.20
5,126,1719AL00,Levanger sentrum Havna renseanlegg,Kjemisk,398.1,36300.0,17700.0,0.94,0.20
6,121,1638AL32,Gammelosen renseanlegg,"Mekanisk - sil, rist",7698.7,30521.4,14000.0,0.03,0.15


In [6]:
gdf = teo.io.get_regine_geodataframe(eng, year)
gdf["vassom"] = gdf["regine"].str.split(".", n=1, expand=True)[0]
vassom_list = [f"{vassom:03d}" for vassom in vassoms]
gdf = gdf.query("vassom in @vassom_list")

# Get main Trøndelag kommune linked to vassoms
kom_list = [int(i) for i in sorted(list(gdf["komnr"].unique())) if i.startswith("50")]
kom_list

[5001,
 5006,
 5021,
 5022,
 5026,
 5027,
 5028,
 5029,
 5031,
 5032,
 5033,
 5034,
 5035,
 5036,
 5037,
 5038,
 5041,
 5053,
 5054,
 5058,
 5059]

In [7]:
# Mapping for SSB codes => TEOTIL3 codes for små anlegg
ssb_csv = "/home/jovyan/projects/teotil3/data/ssb_sma_anlegg_type_codes.csv"
ssb_df = pd.read_csv(ssb_csv)

# Treatment efficiencies for små anlegg
ren_df = pd.read_csv(r"../data/renseeffekter_sma_anlegg.csv", sep=";").set_index(
    "type"
)[["eff_p", "eff_n"]]

sma_csv = r"/home/jovyan/shared/common/teotil3/point_data/historic_data_delivered_2023/tmp_teotil_smaa_anlegg_2002_2022.sdv"
sma_df = pd.read_csv(sma_csv, sep=";", encoding="cp1252").query("aargang == @year")
cols = ["KOMMUNE_NR"] + ssb_df["ssb_code"].tolist()
sma_df = sma_df[cols].query("KOMMUNE_NR in @kom_list").set_index("KOMMUNE_NR")
sma_df = sma_df.sum()
sma_df = pd.concat(
    [sma_df, ssb_df[["ssb_code", "teotil_type"]].set_index("ssb_code")], axis="columns"
).set_index("teotil_type")
sma_df.columns = ["n_people"]
sma_df = sma_df.sort_values("n_people", ascending=False)
sma_df = pd.concat([sma_df, ren_df], axis='columns')
sma_df = sma_df.fillna(100).astype(int)
sma_df.columns = ['Antall personer', 'Renseeffekt TOTP (%)', 'Renseeffekt TOTN (%)']
sma_df

,Antall personer,Renseeffekt TOTP (%),Renseeffekt TOTN (%)
Slamavskiller,17537,5,5
Infiltrasjonsanlegg,16276,75,20
Sandfilteranlegg,12731,15,15
Annen løsning,3550,50,20
Biologisk og kjemisk,1853,90,15
Tett tank for svartvann,657,75,90
Biologisk,621,15,10
Direkte utslipp,375,0,0
Tett tank (for alt avløpsvann),184,100,100
"Tett tank for svartvann, gråvannsfilter",87,90,90
